# Preparation for starting the business

## Business problem
Before starting to create your own business, you should explore lots of different aspects to make the most profitable business in your city. For example, imagine that you would like to create your own little coffee shop in your city (I am from Minsk, Belarus). And one of the first question that you will ask yourself is: "What is the most suitable place for my little coffee shop?".
And this little task of data analize will help you to answer that question.

## Technical Notes
1. The script will use [Foursquare services](https://foursquare.com/) to get the input data (categories and venues)
2. As *Foursquare* gives us only 100 venues per request, I will make several requests based on the districts of my city. And to get district's coordinates, `geopy` library will be used.
3. To show the resulting map with the coffee shop venues, `folium` library will be used.

## Code

### Setup Environment

In [1]:
!pip install pandas geopy folium requests

     |████████████████████████████████| 94 kB 5.3 MB/s  eta 0:00:01


### Import required libraries

In [3]:
import logging
import pandas as pd
import requests
import folium

from geopy.geocoders import Nominatim
from urllib.parse import urlencode
from typing import List, Union

### Create logger

In [4]:
logging.basicConfig(
    format="[%(asctime)s] -- [%(levelname)s]: `%(message)s`",
    datefmt='%H:%M:%S',
    level=logging.INFO
)

### Create global variables

In [5]:
## Foursquare API settings
# Creditials
CLIENT_ID = 'XLREGKRHY3HWJ4MZRAEXVCCRJV3VPUGUG51ZD4J05JH31FLQ'
CLIENT_SECRET = 'T0YUEVV30V3RHKPKI35DOR2EZSM4M1XFQS4LIL0TUSR4U0AN'
VERSION = '20210328'

# Basic API url
BASE_FOURSQUARE_URL = 'https://api.foursquare.com/v2/venues/'

### Create general `fetch` function
This function is going to be one entrypoint for any request to diffrent APIs.
**SPOILER**: I have not use any method except `GET` 😅

In [6]:
def fetch(url: str, method: Union['GET', 'POST', 'PUT', 'DELETE'] = 'GET', params: dict={}):
    """
    Default function for fetching data from different APIs.
    Params:
        :param url: Requsted URL without params.
        :param method: HTTP method for the request. Default `GET`.
        :param params: Params for the request. 
                       If method `GET` => URL params, 
                       any other method => POST params.
    Return:
        :return: Response.
    """
    
    # Check for the valid method
    try:
        request_func = getattr(requests, method.lower())
    except AttributeError:
        logging.error(f'There is no such method as `{method}`. Use `GET`, `POST`, `PUT`, `DELETE` instead.')
        return 

    # If method `GET`, use params as GET params
    if method.lower() == 'get':
        return request_func(
            url=url + '?' + urlencode(params)
        )
    
    # If method not `GET`, use params as `data`
    return request_func(
        url=url,
        data=params
    )

### Create `parse_foursquare_categories` function
This function will fetch all the categories from the *Foursquare* and create `DataFrame` with all category IDs and Names

In [8]:
def parse_foursquare_categories(category_json: List[dict]) -> pd.DataFrame:
    """
    Parse all categories from `foursquare` and create Dataframe based on this data.
    Params:
        :param category_json: Foursquare json with categories.
    Return:
        :return: Dataframe of two columns (ID, Name)
    """
    final_categories_list = []
    
    def get_sub_categories(main_category: dict):
        """
        Check for the sub-categories in main-category. 
        If they are found => run this function recursivly to find sub-sub-categories.
        If no sub-categories => append them to `final_categories_list`.
        Params:
            :param main_category: Main category.
        """
        nonlocal final_categories_list

        for category in main_category['categories']:
            get_sub_categories(category)

        final_categories_list.append(
            (main_category['id'], main_category['name'])
        )


    for category in category_json:
        get_sub_categories(category)

    return pd.DataFrame(data=final_categories_list, columns=['ID', 'Name'])

### Create `get_category_id_by_name` function
This function will find valid *foursquare* id based on the category name 

In [9]:
def get_category_id_by_name(category_name: str) -> Union[str, None]:
    """
    Parse category ID from `foursquare` site.
    Params:
        :param category_name: Name of the category from `foursquare`.
    Return:
        :return: Id of the category or `None`
    """

    # Fetch all categories from `foursquare` API
    resp = fetch(
        BASE_FOURSQUARE_URL + 'categories',
        params={
            'client_id': CLIENT_ID,
            'client_secret': CLIENT_SECRET,
            'v': VERSION
        }
    )

    categories_df = parse_foursquare_categories(resp.json()['response']['categories'])

    # Find valid category by comparing the names
    for index, row in categories_df.iterrows():
        if row['Name'] == category_name:
            break  # `category` variable will store the last category before break 
    else:
        logging.error('There is no such category!')
        return

    return row['ID']

### Create `get_all_the_category_venues` function
This function will request the venues from *Foursquare* via specific category

In [10]:
def get_all_the_category_venues(category_id: str, location: 'geopy.location', radius: int, limit: int = 100) -> pd.DataFrame:
    """
    Find all valid venues from the `foursquare` for current `location` and `radius`.
    Params:
        :param category_id: Foursquare category id.
        :param location: Location of the specific place where venues are going to be found.
        :param radius: Radius of the area, where venues are going to be found.
        :param limit: Limit of venues that are going to be fetched. Default is 100 and it is limit for the foursquare free api.
    Return:
        :return: `DataFrame` with venues (name, lat, lng) 
    """
    resp = fetch(
        BASE_FOURSQUARE_URL + 'search',
        params={
            'categoryId': category_id,
            'll': f'{location.latitude},{location.longitude}',
            'radius': radius,
            'limit': limit,
            'client_id': CLIENT_ID,
            'client_secret': CLIENT_SECRET,
            'v': VERSION
        }
    )

    valid_venues = [(venue['name'].capitalize(), venue['location']['lat'], venue['location']['lng']) for venue in resp.json()['response']['venues']]
        
    return pd.DataFrame(valid_venues, columns=['Name', 'Latitude', 'Longitude'])

### Main Script
What is going to happend:
1. Get all Minsk districts
2. Get `category_id` of the category
3. Go through all districts and fetch venues
4. Concat all the venues
5. Create map with all the venues on it

In [17]:
geolocator = Nominatim(user_agent='final-task')

# STEP 1
# All Minsk districts
districts = [
    "Минск", 
    "Первомайский район, Минск",
    "Советский район, Минск",
    "Центральный район, Минск",
    "Фрунзенский район, Минск",
    "Московский район, Минск",
    "Октябрьский район, Минск",
    "Ленинский район, Минск",
    "Партизанский район, Минск",
    "Заводской район, Минск",
]

# STEP 2
category_id = get_category_id_by_name('Coffee Shop')

# STEP 3
all_venues = []
for district in districts:
    location = geolocator.geocode(district)
    venues_df = get_all_the_category_venues(category_id, location, 2000, 250)
    all_venues.append(venues_df)

# STEP 4
all_venues_df = pd.concat(all_venues)  # Concat all venues from all districts
all_venues_df.reset_index(drop=True, inplace=True)

# STEP 5
m = folium.Map(location=[location.latitude, location.longitude])

for _, row in all_venues_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color="blue", icon="glyphicon glyphicon-record"),
    ).add_to(m)

### Results

In [20]:
# Show all the venues
all_venues_df

,Name,Latitude,Longitude
0,La crête d'or,53.902100,27.557405
1,Зерно,53.899651,27.559976
2,Tiden,53.913045,27.567588
3,32.08,53.914820,27.563395
4,"Кофейня-кондитерская ""валенка""",53.900220,27.554515
...,...,...,...
281,Moya lubimaya kofeinya,53.895280,27.653720
282,Hata,53.912347,27.678615
283,Кофейня на дому,53.895757,27.653318
284,Hata магазин булочек,53.912392,27.678604


In [15]:
# Show final map
m

## Conclusion
I created the script that can look through any city in the world and create map with specific venues. It will help you to choose the most profitable place for your future business.